In [1]:
import os
from experiment_setup import setups

In [38]:
# download the dataset, the links can be fund in the README
dataset_path = "../datasets/coveo_ecommerce"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [39]:
%run ../Preprocess/coveo_ecommerce_preproc.py --path $dataset_path

76324564 5316128 17984974 206876


C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\rees46_preproc.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  view_dedup.drop(


68702669 3376235 10363079 203604


C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\rees46_preproc.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  view_dedup.drop(


68636808 3376094 10362640 172960


...

68630512 3374624 10356870 172825


C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\rees46_preproc.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  view_dedup.drop(


68630512 3374624 10356870 172825


C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\rees46_preproc.py:173: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.min()).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\rees46_preproc.py:178: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt.datetime.utcfromtimestamp(data.Time.max()).strftime(
C:\Users\EVILAB\Documents\RecSys-Project-2024-1\Preprocess\rees46_preproc.py:173: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in

                                             Dataset  NumEvents  NumSessions  \
0    rees46\rees46_processed_view_userbased_full.tsv   68630512     10356870   
1    rees46\rees46_processed_view_userbased_test.tsv    1054210       166841   
2  rees46\rees46_processed_view_userbased_train_f...   67575203     10190006   
3  rees46\rees46_processed_view_userbased_train_t...   66485029     10007176   
4  rees46\rees46_processed_view_userbased_train_v...    1089021       182789   

   NumItems    NumDays                   StartTime  \
0    172825  60.999977  2019-10-01 00:00:00.000000   
1     79894   0.999954  2019-11-30 00:00:02.000000   
2    172756  59.999977  2019-10-01 00:00:00.000000   
3    172430  58.999988  2019-10-01 00:00:00.000000   
4     80618   0.999641  2019-11-29 00:00:29.000000   

                      EndTime  AvgItemViews  MinSessionLength  \
0  2019-11-30 23:59:58.000000    397.109863                 2   
1  2019-11-30 23:59:58.000000     13.195109                 2   

## Use a specific setup for your dataset

In [4]:
params = setups["coveo"]["params_bprmax"]

In [5]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [6]:
def create_gru4rec_pytorch_script(model_name, train_folder, train_data, test_data, model_path, loss, optim, final_act, layers, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, n_epochs, m, eval_hidden_reset, use_correct_loss, use_correct_mask_reset):
    checkpoint_dir = f"{model_path}\\{model_name}"
    s_train_full = (
        f"python ..\\GRU4REC-pytorch\\main.py --data_folder {train_folder} "
        f"--train_data {train_data} --valid_data {test_data} --checkpoint_dir {checkpoint_dir} "
        f"--num_layers 1 --embedding_dim {layers} --hidden_size {layers} "
        f"--loss_type {'BPR-max' if loss == 'bpr-max' else 'CrossEntropy'} --final_act {final_act} "
        f"--n_epochs {n_epochs} --batch_size {batch_size} --dropout_input {dropout_p_embed} "
        f"--dropout_hidden {dropout_p_hidden} --lr {learning_rate} --momentum 0.0 "
        f"--optimizer_type {'Adagrad' if optim == 'adagrad' else ''}"
        f"{' --eval_hidden_reset' if eval_hidden_reset else ''}"
        f"{' --use_correct_loss' if use_correct_loss else ''}"
        f"{' --use_correct_mask_reset' if use_correct_mask_reset else ''}"
    )
    s_test_full = s_train_full + f" --is_eval --load_model {checkpoint_dir}\\model_0000{n_epochs-1}.pt --m {m}"
    return s_train_full, s_test_full

In [7]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [8]:
train_folder, train_data = '/'.join(train_path.split('/')[:-1]), train_path.split('/')[-1]
test_folder, test_data = '/'.join(test_path.split('/')[:-1]), test_path.split('/')[-1]

In [9]:
train_script_oob, test_script_oob = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_oob_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=False, use_correct_loss=False, use_correct_mask_reset=False)

In [10]:
print(train_script_oob)
print(test_script_oob)

python ..\GRU4REC-pytorch\main.py --data_folder ../datasets --train_data coveo_ecommerce\coveo_processed_view_train_full.tsv --valid_data coveo_ecommerce\coveo_processed_view_test.tsv --checkpoint_dir ../trained_models\gru4rec_pytorch_oob_bprmax --num_layers 1 --embedding_dim 512 --hidden_size 512 --loss_type BPR-max --final_act elu-1 --n_epochs 5 --batch_size 144 --dropout_input 0.0 --dropout_hidden 0.0 --lr 0.05 --momentum 0.0 --optimizer_type Adagrad
python ..\GRU4REC-pytorch\main.py --data_folder ../datasets --train_data coveo_ecommerce\coveo_processed_view_train_full.tsv --valid_data coveo_ecommerce\coveo_processed_view_test.tsv --checkpoint_dir ../trained_models\gru4rec_pytorch_oob_bprmax --num_layers 1 --embedding_dim 512 --hidden_size 512 --loss_type BPR-max --final_act elu-1 --n_epochs 5 --batch_size 144 --dropout_input 0.0 --dropout_hidden 0.0 --lr 0.05 --momentum 0.0 --optimizer_type Adagrad --is_eval --load_model ../trained_models\gru4rec_pytorch_oob_bprmax\model_00004.pt -

In [11]:
checkpoint_dir = "../trained_models/gru4rec_pytorch_oob_bprmax"
eval_hidden_reset=False
use_correct_loss=False
use_correct_mask_reset=False

In [13]:
import subprocess

# Comando de entrenamiento
process_train = subprocess.Popen(train_script_oob, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_train, stderr_train = process_train.communicate()

print("Salida de STDOUT (Entrenamiento):")
print(stdout_train.decode())
print("Salida de STDERR (Entrenamiento):")
print(stderr_train.decode())

# Comando de evaluación
process_test = subprocess.Popen(test_script_oob, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_test, stderr_test = process_test.communicate()

print("Salida de STDOUT (Evaluación):")
print(stdout_test.decode())
print("Salida de STDERR (Evaluación):")
print(stderr_test.decode())


Salida de STDOUT (Entrenamiento):
                      Args                                             Values
0              hidden_size                                                512
1               num_layers                                                  1
2               batch_size                                                144
3            dropout_input                                                0.0
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                               [20]
7           optimizer_type                                            Adagrad
8                final_act                                              elu-1
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum    

In [20]:
from track_model import track_training_C02_emissions

track_training_C02_emissions(train_script_oob, train_folder)

ModuleNotFoundError: No module named 'pkg_resources'

### Train the out-of-the-box model

In [17]:
os.system(train_script_oob)

sh: 1: python: not found


32512

### Test the out-of-the-box model

In [ ]:
os.system(test_script_oob)

## Train & test inference fix model

In [ ]:
train_script_inffix, test_script_inffix = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_inffix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=False, use_correct_mask_reset=False)

### Train the out-of-the-box eval fix model

In [ ]:
os.system(train_script_inffix)

### Test the out-of-the-box eval fix model

In [ ]:
os.system(test_script_inffix)

## Train & test the major fix model

In [ ]:
train_script_majorfix, test_script_majorfix = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_majorfix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=True, use_correct_mask_reset=True)

### Train the major fix model

In [ ]:
os.system(train_script_majorfix)

### Test the major fix model

In [ ]:
os.system(test_script_majorfix)